In [1]:
import pyLDAvis
import pandas as pd

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


pyLDAvis 는 LDA 모델로부터 topic - term, doc - topic 확률과 term frequency, doc lengths 를 추출하고, 이를 바탕으로 세 종류의 테이블을 만듭니다. 테이블과 컬럼의 이름이 모호하지만, pyLDAvis 의 변수명을 그대로 따랐습니다.

- topic_coordinates
- topic_info
- token_table

이 세 종류의 테이블을 직접 준비하여 PreparedData 를 만듦으로써 pyLDAvis 를 이해할 수 있습니다.

    class PreparedData:
        def __init__(topic_coordinates, topic_info,
            token_table, R, lambda_step, plot_opts, topic_order):
            ...

### Default value

In [2]:
# default value
R=30
lambda_step=0.01
plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}

### topic info

In [3]:
# topic_info
columns = ['term', 'Category', 'Freq', 'Term', 'Total', 'loglift', 'logprob']

rows = [
    (0, 'Default', 298, '흔한단어', 300, 30, 30),
    (1, 'Default', 100, '공연', 102, 22.3, 22.3),
    (2, 'Default', 95, '무대', 98, 22, 22),
    (3, 'Default', 92, '유명인사', 95, 21.5, 21.5),
    (10, 'Default', 55, '정치인', 58, 18, 18),
    (11, 'Default', 50, '정책', 51, 17, 17),
    (20, 'Default', 81, '박물관', 82, 19, 19),
    (21, 'Default', 17, '큐레이터', 18, 15, 15),
    (22, 'Default', 18, '명화', 18.1, 15.5, 15.5),

    (0, 'Topic1', 150, '흔한단어', 300, 5, -5),
    (3, 'Topic1', 55, '유명인사', 98, 4, -4),
    (1, 'Topic1', 50, '공연', 102, 3.8, -4.4),
    (2, 'Topic1', 45, '무대', 98, 3.7, -4.1),

    (0, 'Topic2', 100, '흔한단어', 300, 3, -5),
    (3, 'Topic2', 25, '유명인사', 98, 3, -4),
    (10, 'Topic2', 55, '정치인', 58, 3.4, -3.4),
    (11, 'Topic2', 50, '정책', 51, 3.7, -2.1),

    (0, 'Topic3', 48, '흔한단어', 300, 2, -5),
    (3, 'Topic3', 22, '유명인사', 98, 2, -4),
    (20, 'Topic3', 81, '박물관', 82, 4.4, -3.4),
    (21, 'Topic3', 17, '큐레이터', 18, 3.7, -2.1),
    (22, 'Topic3', 18, '명화', 18.1, 4.7, -2.1),
]

with open('./topic_info.csv', 'w', encoding='utf-8') as f:
    f.write(','.join(columns) + '\n')
    for row in rows:
        f.write('{}\n'.format(','.join(str(v) for v in row)))

topic_info = pd.DataFrame.from_csv('./topic_info.csv')

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [4]:
topic_info

,Category,Freq,Term,Total,loglift,logprob
term,,,,,,
0,Default,298,흔한단어,300.0,30.0,30.0
1,Default,100,공연,102.0,22.3,22.3
2,Default,95,무대,98.0,22.0,22.0
3,Default,92,유명인사,95.0,21.5,21.5
10,Default,55,정치인,58.0,18.0,18.0
11,Default,50,정책,51.0,17.0,17.0
20,Default,81,박물관,82.0,19.0,19.0
21,Default,17,큐레이터,18.0,15.0,15.0
22,Default,18,명화,18.1,15.5,15.5


(실제 데이터 예시)

        Category	Freq	Term	Total	loglift	logprob
    term						
    8537	Topic20	1112.902466	취하고	1115.395142	4.2541	-3.5667
    9034	Topic20	1217.860840	포즈	1220.655029	4.2540	-3.4765
    7590	Topic20	836.985291	제작발표회	839.770264	4.2530	-3.8516

### topic coordinate

In [5]:
# topic_coordinate, topics 는 1 부터 시작
columns = ['topic', 'x', 'y', 'topics', 'cluster', 'Freq']
rows = [
    (3, 1, 1, 1, 1, 3.5),
    (1, -1.5, -0.5, 2, 1, 2.1),
    (2, 0.5, 2, 3, 1, 0.7)
]
topic_coordinate = pd.DataFrame.from_records(rows, columns=columns, index='topic')

In [6]:
topic_coordinate.to_csv('./topic_coordinates.csv')

In [7]:
topic_coordinate_2 = pd.DataFrame.from_csv('./topic_coordinates.csv')

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [8]:
topic_coordinate_2

,x,y,topics,cluster,Freq
topic,,,,,
3,1.0,1.0,1,1,3.5
1,-1.5,-0.5,2,1,2.1
2,0.5,2.0,3,1,0.7


Freq 의 최소값은 0 보다 커야 함. 원의 크기를 결정하는 값. [0.5 ~ 3]으로 범위를 잡아두면 좋음

(실제 데이터 예시)

    x	y	topics	cluster	Freq
    topic					
    87	0.116826	0.192296	1	1	3.327655
    50	0.172420	-0.093400	2	1	3.040433
    95	-0.002153	-0.167669	3	1	2.809797
    81	0.142578	-0.214649	4	1	2.287351
    68	0.103613	0.156019	5	1	2.070516

### token table

In [9]:
# token_table
columns = ['term', 'Topic', 'Freq', 'Term']
rows = [
    (0, 1, 0.3, '흔한단어'),
    (0, 2, 0.4, '흔한단어'),
    (0, 3, 0.3, '흔한단어'),
    (1, 1, 0.95, '공연'),
    (2, 1, 0.98, '무대'),
    (3, 1, 0.55, '유명인사'),
    (3, 2, 0.24, '유명인사'),
    (3, 3, 0.20, '유명인사'),
    (10, 2, 0.85, '정치인'),
    (11, 2, 0.99, '정책'),
    (20, 3, 0.85, '박물관'),
    (21, 3, 0.97, '큐레이터'),
    (22, 3, 0.97, '명화'),
]

with open('./token_table.csv', 'w', encoding='utf-8') as f:
    f.write(','.join(columns)+'\n')
    for row in rows:
        f.write(','.join((str(v) for v in row))+'\n')

In [10]:
token_table = pd.DataFrame.from_csv('./token_table.csv')

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [11]:
token_table

,Topic,Freq,Term
term,,,
0,1,0.30,흔한단어
0,2,0.40,흔한단어
0,3,0.30,흔한단어
1,1,0.95,공연
2,1,0.98,무대
3,1,0.55,유명인사
3,2,0.24,유명인사
3,3,0.20,유명인사
10,2,0.85,정치인


Freq 의 합은 1 보다 작아야 함 (1의 근사치)

(실제 데이터 예시)

    Topic	Freq	Term
    term			
    2429	7	0.016450	뉴스
    2429	8	0.006729	뉴스
    2429	10	0.015702	뉴스

### topic order

In [12]:
# topic_order
topic_order = [3, 1, 2]

In [13]:
prepared_data = pyLDAvis.PreparedData(topic_coordinate, topic_info, token_table, R, lambda_step, plot_opts, topic_order)

In [14]:
pyLDAvis.display(prepared_data)

In [15]:
pyLDAvis.save_html(prepared_data, 'example.html')